In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import Loss
import time 
# from tensorflow.keras.applications import *
# from matplotlib import plotly as plt
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pickle


In [ ]:


def conv_layer_to_weight_mats(in_layer_shape, conv_dict, conv_tensor,
                              max_weight_convention=='all_one'):
    '''
    take in the shape of the incoming layer, a dict representing info about the
    conv operation, and the weight tensor of the convolution, and return an 
    array of sparse weight matrices representing the operation. the array should
    have a single element if layer_dict['max_pool_after']==False, but should have 
    two (one representing the action of the max pooling) otherwise.
    for max pooling, we linearise by connecting the maxed neurons to everything
    in their receptive field. if max_weight_convention=='all_one', all the 
    weights are one, otherwise if max_weight_convention=='one_on_n', the weights
    are all one divided by the receptive field size
    '''
    # TODO: see if vectorisation will work
    kernel_height, kernel_width, n_chan_in, n_chan_out = conv_tensor.shape
    in_height = in_layer_shape[0]
    in_width = in_layer_shape[1]
    assert (kernel_height, kernel_width) == tuple(conv_dict['kernel_size']), f"weight tensor info doesn't match conv layer dict info - kernel size from conv_tensor.shape is {(kernel_height, kernel_width)}, but conv_dict says it's {conv_dict['kernel_size']}"
    assert n_chan_out == conv_dict['filters'], f"weight tensor info doesn't match conv layer dict info: weight tensor says num channels out is {n_chan_out}, conv dict says it's {conv_dict['filters']}"
    assert in_layer_shape[2] == n_chan_in, f"weight tensor info doesn't match previous layer shape: weight tensor says it's {n_chan_in}, prev layer says it's {in_layer_shape[2]}"

    kernel_height_centre = int((kernel_height - 1) / 2)
    kernel_width_centre = int((kernel_width - 1) / 2)

    in_layer_size = np.product(in_layer_shape)
    out_layer_shape = (in_height, in_width, n_chan_out)
    out_layer_size = np.product(out_layer_shape)

    conv_weight_matrix = np.zeros((in_layer_size, out_layer_size))

    # THIS WORKS ONLY FOR SAME and not for VALID!!!
    for i in range(in_height):
        for j in range(in_width):
            for c_out in range(n_chan_out):
                out_int = cnn_layer_tup_to_int((i,j,c_out), out_layer_shape)
                for n in range(kernel_height):
                    for m in range(kernel_width):
                        for c_in in range(n_chan_in):
                            weight = conv_tensor[n][m][c_in][c_out]
                            h_in = i + n - kernel_height_centre
                            w_in = j + m - kernel_width_centre
                            in_bounds_check = (h_in in range(in_height)
                                               and w_in in range(in_width))
                            if in_bounds_check:
                                in_int = cnn_layer_tup_to_int((h_in, w_in,
                                                               c_in),
                                                              in_layer_shape)
                                conv_weight_matrix[in_int][out_int] = weight

    weights_array = [conv_weight_matrix]

    if conv_dict['max_pool_after']:
        k_height, k_width = conv_dict['max_pool_size']
        stride = conv_dict['max_pool_size']  # conv_dict['max_pool_stride']
        padding = conv_dict['max_pool_padding']

        if max_weight_convention == 'all_one':
            max_weight = 1
        elif max_weight_convention == 'one_on_n':
            max_weight = 1 / (k_height * k_width)
        else:
            raise ValueError("max_weight_convention must be 'one_on_n' or 'all_one', is instead" + max_weight_convention)

        # This code works on valid, I tested it
        # But if input_side is divisible by stride, it is the same
        if padding == 'valid':
            maxed_height = math.ceil(in_height / stride[0])
            maxed_width = math.ceil(in_width / stride[1])
            maxed_shape = (maxed_height, maxed_width, n_chan_out)
            maxed_size = np.product(maxed_shape)
            max_matrix = np.zeros((out_layer_size, maxed_size))

            k_height_centre = int((k_height - 1) / 2)
            k_width_centre = int((k_width - 1) / 2)
            
            for i in range(maxed_height):
                for j in range(maxed_width):
                    for c in range(n_chan_out):
                        max_int = cnn_layer_tup_to_int((i,j,c), maxed_shape)
                        for n in range(k_height):
                            for m in range(k_width):
                                h_in = stride[0] * i + n - k_height_centre
                                w_in = stride[1] * j + m - k_width_centre
                                in_bounds_check = (h_in in range(in_height)
                                                   and
                                                   w_in in range(in_width))
                                if in_bounds_check:
                                    out_int = cnn_layer_tup_to_int(
                                        (h_in, w_in, c), out_layer_shape
                                    )
                                    max_matrix[out_int][max_int] = max_weight

        # originally, this was 'valid`, but I don't know what it is
        # this code reaise an IndexError
        elif padding == 'same':
            raise NotImplementedError

            maxed_height = math.ceil((in_height - k_height + 1) / stride[0])
            maxed_width = math.ceil((in_width - k_width + 1) / stride[1])
            maxed_shape = (maxed_height, maxed_width, n_chan_out)
            maxed_size = np.product(maxed_shape)
            max_matrix = np.zeros((out_layer_size, maxed_size))

            for i in range(maxed_height):
                for j in range(maxed_width):
                    for c in range(n_chan_out):
                        max_int = cnn_layer_tup_to_int((i,j,c), maxed_shape)
                        for n in range(kernel_height):
                            for m in range(kernel_width):
                                h_in = stride[0] * i + n
                                w_in = stride[1] * i + m
                                out_int = cnn_layer_tup_to_int((h_in, w_in, c),
                                                               out_layer_shape)
                                max_matrix[out_int][max_int] = max_weight
        else:
            raise ValueError("invalid value for 'max_pool_padding'")

        weights_array.append(max_matrix)

    return weights_array

In [ ]:
def weights_to_graph(weights_array):
    # take an array of weight matrices, and return the adjacency matrix of the
    # neural network it defines.
    # if the weight matrices are A, B, C, and D, the adjacency matrix should be
    # [[0   A   0   0   0  ]
    #  [A^T 0   B   0   0  ]
    #  [0   B^T 0   C   0  ]
    #  [0   0   C^T 0   D  ]
    #  [0   0   0   D^T 0  ]]

    block_mat = []

    # for everything in the weights array, add a row to block_mat of the form
    # [None, None, ..., sparsify(np.abs(mat)), None, ..., None]
    for (i, mat) in enumerate(weights_array):
        sp_mat = sparse.coo_matrix(np.abs(mat))
        if i == 0:
            # add a zero matrix of the right size to the start of the first row
            # so that our final matrix is of the right size
            n = mat.shape[0]
            first_zeroes = sparse.coo_matrix((n, n))
            block_row = [first_zeroes] + [None]*len(weights_array)
        else:
            block_row = [None]*(len(weights_array) + 1)
        block_row[i+1] = sp_mat
        block_mat.append(block_row)

    # add a final row to block_mat that's just a bunch of [None]s followed by a
    # zero matrix of the right size
    m = weights_array[-1].shape[1]
    final_zeroes = sparse.coo_matrix((m, m))
    nones_row = [None]*len(weights_array)
    nones_row.append(final_zeroes)
    block_mat.append(nones_row)

    # turn block_mat into a sparse matrix
    up_tri = sparse.bmat(block_mat, 'csr')

    # we now have a matrix that looks like
    # [[0   A   0   0   0  ]
    #  [0   0   B   0   0  ]
    #  [0   0   0   C   0  ]
    #  [0   0   0   0   D  ]]
    # add this to its transpose to get what we want
    adj_mat = up_tri + up_tri.transpose()
    return adj_mat

In [ ]:
model = tf.load_saved_model(model_path)
weights = []

for layer in model.layers: 
    w = None
    if 'conv' in layer.name: 
        w = conv_layer_to_weight_mats(layer.weights)
    else if 'dense' in layer.name: 
        w = layer.weights
    if w: 
        weights.append(w)
weights_to_graph(weights)

